refer-to: https://www.tensorflow.org/get_started/tensorboard_histograms  
original: https://github.com/tensorflow/tensorboard/blob/master/tensorboard/plugins/histogram/histograms_demo.py

In [ ]:
import tensorflow as tf
import os

In [ ]:
from tensorboard.plugins.histogram import summary as histogram_summary

In [ ]:
# LOG_DIR = '/tmp/histograms_demo'
CUR_DIR = os.path.abspath(os.path.curdir)
LOG_DIR = os.path.join(CUR_DIR, 'histograms_demo')

In [ ]:
tf.set_random_seed(0)
k = tf.placeholder(tf.float32)

In [ ]:
# Make a normal distribution, with a shifting mean
mean_moving_normal = tf.random_normal(shape=[1000], mean=(5*k), stddev=1)
# Record that distribution into a histogram summary
histogram_summary.op("normal/moving_mean",
                     mean_moving_normal,
                     description="時間とともに平均が変化する正規分布。")

In [ ]:
# Make a normal distribution with shrinking variance
shrinking_normal = tf.random_normal(shape=[1000], mean=0, stddev=1-(k))
# Record that distribution too
histogram_summary.op("normal/shrinking_variance", shrinking_normal,
                     description="分散が時間とともに縮小する正規分布。")

In [ ]:
# Let's combine both of those distributions into one dataset
normal_combined = tf.concat([mean_moving_normal, shrinking_normal], 0)
# We add another histogram summary to record the combined distribution
histogram_summary.op("normal/bimodal", normal_combined,
                     description="2つの正規分布の組み合わせ、1つは移動平均を持ち、もう1つは分散が縮小していくもの。"
                                 "結果としては、unimodal で始まり、時間と共にだんだん bimodal になる分布。")

In [ ]:
# Add a gamma distribution
gamma = tf.random_gamma(shape=[1000], alpha=k)
histogram_summary.op("gamma", gamma,
                     description="形状パラメータαが経時的に変化するガンマ分布。")

In [ ]:
# And a poisson distribution
poisson = tf.random_poisson(shape=[1000], lam=k)
histogram_summary.op("poisson", poisson,
                     description="整数値のみをとるポアソン分布。")

In [ ]:
# And a uniform distribution
uniform = tf.random_uniform(shape=[1000], maxval=k*10)
histogram_summary.op("uniform", uniform,
                     description="シンプルな一様分布。")

In [ ]:
# Finally, combine everything together!
all_distributions = [mean_moving_normal, shrinking_normal,
                     gamma, poisson, uniform]
all_combined = tf.concat(all_distributions, 0)
histogram_summary.op("all_combined", all_combined,
                     description="5つの分布の融合：一様分布、ガンマ分布、ポアソン分布、二つの正規分布。")

In [ ]:
summaries = tf.summary.merge_all()

In [ ]:
# Setup a session and summary writer
# sess = tf.Session()
sess = tf.InteractiveSession()
writer = tf.summary.FileWriter(LOG_DIR)

In [ ]:
# Setup a loop and write the summaries to disk
N = 400
for step in range(N):
    k_val = step/float(N)
    summ = sess.run(summaries, feed_dict={k: k_val})
    writer.add_summary(summ, global_step=step)

In [ ]:
writer.close()

run: `tensorboard --logdir=$(pwd)/histograms_demo`  
( or run: `./run_tensorboard_histogram_demo.sh` )